In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
order_1 = pd.read_csv('season_1/training_data/order_data/order_data_2016-01-01', delimiter='\t',header=None)
order_1.columns = ["order_id", "driver_id","passenger_id","start_district_hash", "dest_district_hash", "Price","Time"]
order_1.head(1)

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,Price,Time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24,2016-01-01 13:37:23


In [43]:
def output(day):
    order = pd.read_csv('season_1/training_data/order_data/order_data_{}'.format(day),delimiter='\t',header=None)
    order.columns = ["order_id", "driver_id","passenger_id","start_district_hash", "dest_district_hash", "Price","Time"]

    # 时间信息
    order["time"] = pd.to_datetime(order.Time,unit='s')
    # order["time_id"] = order.time.apply(lambda x: x.hour*6 + int(x.minute/10)+1) #这一行和下一行都可以
    order["time_id"] = (order.time - datetime.date(order.time).dt.seconds/600).astype(np.int64) + 1 
    # 日期信息  
    order["weekday"] = datetime.isoweekday(datetime.date(order.time))
    order["date_id"] = np.where(order.weekday == (6 or 7), 1,0)
    if datetime.date(order.time) == datetime.date(2016, 1, 1):
        order["date_id"] = 1
    # 地点信息
    district_data = pd.read_csv('season_1/training_data/cluster_map/cluster_map', delimiter='\t',header=None)
    district_data.columns = ["start_district_hash","district_id"]
    # 合并地区信息
    order_m = pd.merge(order,district_data,on="start_district_hash")
    # 简化表格，并计算gap
    std_df = order_m[["driver_id", "date_id", "time_id","district_id"]]
    # 整理 driver_id 信息，将 非NaN 都变为0，将NaN变为1
    mask = std_df.isnull()
    # convert the NaN data (gap) to 1 
    std_df['gap'] = np.where(mask.driver_id,1,0)
    # rid the hash data
    std_df =std_df.drop("driver_id",axis=1)
    # 
    std_output = std_df.groupby(["district_id","time_id"]).sum()
   
    std_output.to_csv('order_data_{}_gap.csv'.format(day),index=True)